## CONVOLUTIONAL NEURAL NETWORK MODEL FOR WASTE CASSIFICATION

## 1. Library Imports and Environment Setup

In [ ]:
# -*- coding: utf-8 -*-
"""Image classification optimized to use more RAM"""

import os
import zipfile
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import mixed_precision

import warnings
warnings.filterwarnings('ignore')

#  Enable Mixed Precision to reduce VRAM consumption
mixed_precision.set_global_policy('mixed_float16')

#  Configure GPU to allow dynamic memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(" GPU configurada con crecimiento de memoria dinámico")
    except RuntimeError as e:
        print(e)

## 2. Dataset Loading and Verification

In [ ]:

#  Dataset configuration
data_dir = pathlib.Path("/home/daniel/Documents/SIGR/SIRG DATASET/train")
if not data_dir.exists():
    raise FileNotFoundError(f"ERROR: no such directory: {data_dir}")

#  Image parameters
batch_size = 20  #  increase batch size to use more RAM
img_height = 400
img_width = 400
AUTOTUNE = tf.data.AUTOTUNE

#  Load datasets with higher RAM usage
print(" loading TRAINING DATASET...")
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.1,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

print("\n loading TESTING DATASET...")
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.1,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

#  Obtener clases
class_names = train_ds.class_names
print("\n CLASSES:", class_names)


## 3. Data Preprocessing and Normalization

In [ ]:


#  Configure dataset for performance **(more ram usage)**
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)

train_ds = train_ds.cache().shuffle(2000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## 4. Data Augmentation Strategy

In [ ]:

#  Data augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


## 5. Optimized CNN Model Architecture

In [ ]:

#  Optimized model
num_classes = len(class_names)
model = Sequential([
    data_augmentation,
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Deletes 30% of the neurons to prevent overfitting
    layers.Dense(num_classes),

])

#  Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.build(input_shape=(None, img_height, img_width, 3))
model.summary()


## 6. Model Compilation

In [ ]:


num_batches = tf.data.experimental.cardinality(train_ds).numpy()
print(f" Total number of batches in train_ds: {num_batches}")

for image_batch, label_batch in train_ds.take(1):  # Take one batch for demonstration
    print(f" Batch size: {image_batch.shape}") 


## 7. Model Training Process

In [ ]:

#  Model training
epochs = 40
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)


## 8. Model Evaluation and Metric Visualization

In [ ]:

#  Save the model
model.save('/home/daniel/Documents/SIGR/model_ver1.1.keras')

#  Convert model to TensorFlow Lite (optimizado para float16)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.allow_custom_ops = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.SELECT_TF_OPS]  #  Habilita TF Select
tflite_model = converter.convert()

#  Save model in .tflite format
tflite_path = "/home/daniel/Documents/SIGR/model_ver1.1.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f" Modelo guardado en: {tflite_path}")


## 9. Saving the Trained Model

In [ ]:

#  Function for image prediction
def predict_image(image_path):
    if not os.path.exists(image_path):
        print(f"X Image not found: {image_path}")
        return

    img = keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
    img_array = keras.preprocessing.image.img_to_array(img)

    # Normalize image (as in training)
    img_array = img_array / 255.0  # Normalizar a rango [0,1]

    img_array = tf.expand_dims(img_array, 0)  # Create batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(f"🔍 The image is probably {class_names[np.argmax(score)]} with a confidence of {100 * np.max(score):.2f}%.")


In [ ]:

#  Prediction test with an image
test_image_path = "/home/daniel/Documents/SIGR/Images/test/camisa.jfif"
predict_image(test_image_path)


## 10. Results Visualization

In [ ]:

#  Visualize training results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
